<a href="https://colab.research.google.com/github/ranjanguddu/EPAiPhase1/blob/master/session-10/session-10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install faker

In [3]:
from faker import Faker
from datetime import date, datetime, timezone
from time import perf_counter, localtime
from functools import wraps
import random
import pandas as pd
from time import perf_counter
from collections import namedtuple
from collections import defaultdict

In [4]:
fake = Faker(['it_IT','en_IN'])
random_profile = namedtuple('Profile', ['name', 'age', 'sex', 'blood_group', 'location'])


# calculate the age of the person
find_age = lambda birthdate: int((date.today() - birthdate).days / 365)

# REPEAT DECORATOR
def repeat(n: int):
	"""
	This decorator is suppose to time the function. 
	"""
	def timed(fn):
		def inner(*args, **kwargs):
			total_elapsed = 0
			
			for _ in range(n):
				start = perf_counter()
				result = fn(*args, **kwargs)
				total_elapsed += (perf_counter() - start)

			avg_run_time = total_elapsed / n
			print(f'Average Run time of {fn.__name__} is : {avg_run_time} for {n} reps.')
			return result
		return inner
	return timed


profile_list = []

@repeat(10)
def generate_random_profile(n: int):
	"""
	This function suppose to generate n no of random profile using named Dictionary
	"""
	blood_type = defaultdict(lambda : 0)
	#print(f'blood type:{blood_type}')
	for _ in range(n):
		prof = fake.profile()
		profile = random_profile(prof['name'], find_age(prof['birthdate']), prof['sex'], prof['blood_group'], prof['current_location'])
		profile_list.append(profile)
		blood_type[profile.blood_group] +=1
		
	av_age = sum([i.age for i in profile_list])/len(profile_list)
	oldest = max([i.age for i in profile_list])
	mean_loaction = sum([i.location[0] for i in profile_list])/len(profile_list), sum([i.location[1] for i in profile_list])/len(profile_list)
	largest_blood_group = sorted(blood_type.items(), key=lambda x: x[1], reverse=True)[0]
	return f'average age:{av_age} years,oldest one is of {oldest} years, mean location:{mean_loaction}, largest_blood:{largest_blood_group}'

res = generate_random_profile(1000)

Average Run time of generate_random_profile is : 1.9618635177000043 for 10 reps.


In [5]:
profile_list_dict = []
blood_type = defaultdict(lambda: 0)


@repeat(10)
def generate_random_profile_dicts(n: int):
	"""
	This function suppose to generate n no of random profile using Dictionary
	"""
	for _ in range(n):
		profile_dict =  defaultdict(lambda :0)
		fake_prof = fake.profile()
		attribue = ['name', 'birthdate', 'sex', 'blood_group', 'current_location']
		for key in attribue:
			#print("hello")
			profile_dict[key] = fake_prof[key]
			
		age = find_age(fake_prof['birthdate'])
		profile_dict['age'] = age

		blood_type[profile_dict['blood_group']] += 1
		profile_list_dict.append(profile_dict)

	avg_age = sum([item['age']for item in profile_list_dict])/len(profile_list_dict)
	max_age = max([item['age']for item in profile_list_dict])
	mean_loc = sum([item['current_location'][0] for item in profile_list_dict])/len(profile_list_dict), sum([item['current_location'][1] for item in profile_list_dict])/len(profile_list_dict) 
	#print(type(mean_loc))
	return f'Average age:{avg_age} years.\n Oldest person: {max_age} years\nThe loaction:({mean_loc} Degrees.\
			 \nLargest Blood Category:{sorted(blood_type.items(), key=lambda x: x[1], reverse=True)[0]}'

	#print(profile_list_dict)
res = generate_random_profile_dicts(1000)
print(res)

Average Run time of generate_random_profile_dicts is : 1.9727910967000013 for 10 reps.
Average age:57.4113 years.
 Oldest person: 116 years
The loaction:((Decimal('1.04972898555'), Decimal('0.1067819322')) Degrees.			 
Largest Blood Category:('B-', 1301)


In [6]:
symbol_generator =  lambda x:x[:3]+'_SYMB'.upper()
companies = namedtuple('StockMarket', ['name', 'symbol', 'open', 'low', 'high', 'close'])
weight = []
company_stock_profiles = []

def stock_exchange(n):
	""" This function suppose to generate n no of company's fake profile """
	
	for _ in range(n):
		name =  fake.company()
		symbol= symbol_generator(name)
		#print(f'{name}:{symbol}')
		open_value = random.randint(100, 4000)
		comp_weight = random.uniform(0, 0.8)

		weight.append(comp_weight)

		comp_contrib = round(open_value * comp_weight/sum(weight), 3)

		high_value = round(random.uniform(1.0, 1.2) * open_value * comp_weight/sum(weight), 3)
		close_value = round(random.uniform(0.6,0.8) * open_value * comp_weight/sum(weight), 3)
		low_value = round(random.uniform(0.2, 0.4) * open_value * comp_weight/sum(weight), 3)

		comp_profile = companies(name, symbol, open_value, low_value, high_value, close_value)
		#print(comp_profile)
		company_stock_profiles.append(comp_profile)
		Total = pd.DataFrame({
					'Open': ['= '+str(round(sum([i.open for i in company_stock_profiles]), 3))], 
					'Low': ['= '+str(round(sum([i.low for i in company_stock_profiles]), 3))],
					'High': ['= '+str(round(sum([i.high for i in company_stock_profiles]), 3))],
					'Close': ['= '+str(round(sum([i.close for i in company_stock_profiles]), 3))]
							})

		stock = pd.DataFrame({'Company Name': [i.name for i in company_stock_profiles],
								'Symbol': [i.symbol for i in company_stock_profiles],
								'Open': [i.open for i in company_stock_profiles], 
								'Low': [i.low for i in company_stock_profiles],
								'High': [i.high for i in company_stock_profiles],
								'Close': [i.close for i in company_stock_profiles]
								})
	return stock.append(Total)
		

res = stock_exchange(5)
print(res)

             Company Name    Symbol    Open        Low        High       Close
0              Moneta SPA  Mon_SYMB     302     65.097     360.836     214.135
1          Evans and Sons  Eva_SYMB    1546     59.812     299.412     172.335
2            Toso e figli  Tos_SYMB    3249    174.602     893.316     641.357
3  Lettiere-Piane e figli  Let_SYMB     512     18.858       94.31      51.505
4                Moss PLC  Mos_SYMB    3098    238.915     819.457     588.629
0                     NaN       NaN  = 8707  = 557.284  = 2467.331  = 1667.961
